In [ ]:
import numpy as np 
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout,BatchNormalization,SpatialDropout1D

Load Data & Split Data to train&validiation

In [ ]:
#Load your data
x_all = np.load('X_all.npy')
y_all = np.load('y_all.npy')
x_train, x_val, y_train, y_val = train_test_split(x_all, y_all,test_size=0.20,random_state=69)

1D-CNN Model

In [ ]:
#CNN
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout,BatchNormalization,SpatialDropout1D
# Create sequential model 
cnn_model = tf.keras.models.Sequential()
#Zero CNN layer  with 32 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=32, kernel_size=(20,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001), input_shape = (400, 20)))
cnn_model.add(BatchNormalization())
#First CNN layer  with 32 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=32, kernel_size=(20,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
cnn_model.add(BatchNormalization())
cnn_model.add(SpatialDropout1D(0.5))
#Second CNN layer  with 64 filters, conv window 3, relu activation and same padding
cnn_model.add(Conv1D(filters=32, kernel_size=(6,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
cnn_model.add(BatchNormalization())
#Third CNN layer with Max pooling
cnn_model.add(MaxPool1D(pool_size=(2,), strides=2, padding='same'))
cnn_model.add(Conv1D(filters=32, kernel_size=(3,), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
cnn_model.add(SpatialDropout1D(0.5))
#Flatten the output
cnn_model.add(Flatten())
#Add a dense layer with 296 neurons
cnn_model.add(Dense(units = 296, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Add a dense layer with 148 neurons
cnn_model.add(Dense(units = 148, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
#Add a dense layer with 74 neurons
cnn_model.add(Dense(units = 74, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))

#Softmax as last layer with five outputs
cnn_model.add(Dense(units = 4, activation='softmax'))

loss = tf.keras.losses.categorical_crossentropy
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

cnn_model.compile(optimizer=optimizer, loss = loss, metrics=['accuracy'])
cnn_model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
modelPath = os.path.join(os.getcwd(), 'bestModel.h5')
checkpoint = ModelCheckpoint( # set model saving checkpoints
    modelPath, # set path to save model weights
    monitor='val_loss', # set monitor metrics
    verbose=1, # set training verbosity
    save_best_only=True, # set if want to save only best weights
    save_weights_only=False, # set if you want to save only model weights
    mode='auto', # set if save min or max in metrics
    period=1 # interval between checkpoints
    )

earlystopping = EarlyStopping(
    monitor='val_loss', # set monitor metrics
    min_delta=0.001, # set minimum metrics delta
    patience=4, # number of epochs to stop training
    restore_best_weights=True, # set if use best weights or last weights
    )
callbacksList = [checkpoint, earlystopping] # build callbacks list

cnn_model_history = cnn_model.fit(x_train, y_train, epochs=100, batch_size = 64, validation_data = (x_val, y_val), callbacks=callbacksList)


Load testdata

In [ ]:
#load your testdata 
testdata = np.load('test.npy')

#Prediction
ypred = cnn_model.predict(testdata)
sub=[]
for i in ypred:
  sub.append(np.argmax(i)) 

Submission on kaggle

In [ ]:
import pandas as pd
a=pd.read_csv('sample_submission.csv')

num=[]
for i in range(400):
  num.append(i+1)
for i in range(400):
  num.append('f'+str(i+1))
for i in range(400):
  sub.append(None)
import pandas as pd
df = pd.DataFrame({'sample_id':num , 'prediction':sub})
df.to_csv('submission.csv',index=False)
df = df.reset_index(drop=True)